### Aplicação da validação cruzada para a base de carros

#### Importar base de dados que foi processada anteriormente

In [1]:
import pandas as pd

In [2]:
base = pd.read_csv('../../data/baseCarrosProcessada.csv', encoding = 'ISO-8859-1')

In [3]:
base[:10]

,Unnamed: 0,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,5,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja
6,6,2200,test,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein
7,8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,nein
8,9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,benzin,volkswagen,nein
9,10,2000,control,limousine,2004,manuell,105,3_reihe,150000,12,benzin,mazda,nein


### Remover a coluna com o index antigo

In [4]:
base = base.drop('Unnamed: 0', axis=1)

In [5]:
base[:10]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
5,650,test,limousine,1995,manuell,102,3er,150000,10,benzin,bmw,ja
6,2200,test,cabrio,2004,manuell,109,2_reihe,150000,8,benzin,peugeot,nein
7,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,nein
8,999,test,kleinwagen,1998,manuell,101,golf,150000,0,benzin,volkswagen,nein
9,2000,control,limousine,2004,manuell,105,3_reihe,150000,12,benzin,mazda,nein


### Extrair dados para treinamento predição e os dados da classe (preço real)

In [6]:
previsores = base.iloc[:, 1:13].values

In [7]:
preco_real = base.iloc[:, 0].values

### Transformação dos dados

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelEncoder_previsores = LabelEncoder()

In [9]:
previsores[:,0] = labelEncoder_previsores.fit_transform(previsores[:,0])
previsores[:,1] = labelEncoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = labelEncoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = labelEncoder_previsores.fit_transform(previsores[:,5])
previsores[:,8] = labelEncoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = labelEncoder_previsores.fit_transform(previsores[:,9])
previsores[:,10] = labelEncoder_previsores.fit_transform(previsores[:,10])

### Categorização

In [11]:
from sklearn.compose import ColumnTransformer

In [12]:
ct = ColumnTransformer([("", OneHotEncoder(),[0,1,3,5,8,9,10])], remainder="passthrough") # The last arg ([0]) is the list of columns you want to transform in this step
previsores = ct.fit_transform(previsores)

In [13]:
previsores.shape

(359291, 316)

### Criação da Rede

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasRegressor

In [16]:
def criarRede():
    regressor = Sequential()
    regressor.add(Dense(units= 158, activation='relu', input_dim=316))
    regressor.add(Dense(units= 158, activation='relu'))
    regressor.add(Dense(units=1, activation='linear'))
    regressor.compile(loss = 'mean_absolute_error', optimizer='adam', metrics = ['mean_absolute_error'])
    return regressor

#### Treinamento do modelo com validação cruzada

In [17]:
regressor = KerasRegressor(build_fn = criarRede, epochs = 100, batch_size = 300)
resultados = cross_val_score(estimator = regressor, X = previsores, y = preco_real,
                             cv = 10, scoring = 'neg_mean_absolute_error')

Epoch 1/100
323361/323361 [==============================] - 10s 32us/step - loss: 3874.3246 - mean_absolute_error: 3874.3242
Epoch 2/100
323361/323361 [==============================] - 11s 33us/step - loss: 3379.0888 - mean_absolute_error: 3379.0894
Epoch 3/100
323361/323361 [==============================] - 10s 30us/step - loss: 3096.3806 - mean_absolute_error: 3096.3828
Epoch 4/100
323361/323361 [==============================] - 9s 29us/step - loss: 2930.0067 - mean_absolute_error: 2930.0061
Epoch 5/100
323361/323361 [==============================] - 11s 35us/step - loss: 2836.2354 - mean_absolute_error: 2836.2363
Epoch 6/100
323361/323361 [==============================] - 11s 35us/step - loss: 2779.0137 - mean_absolute_error: 2779.0127
Epoch 7/100
323361/323361 [==============================] - 11s 33us/step - loss: 2738.5388 - mean_absolute_error: 2738.5403
Epoch 8/100
323361/323361 [==============================] - 13s 41us/step - loss: 2690.0946 - mean_absolute_error: 269

323362/323362 [==============================] - 10s 30us/step - loss: 2381.1362 - mean_absolute_error: 2381.1355
Epoch 30/100
323362/323362 [==============================] - 11s 35us/step - loss: 2377.1246 - mean_absolute_error: 2377.1250
Epoch 31/100
323362/323362 [==============================] - 11s 34us/step - loss: 2363.4380 - mean_absolute_error: 2363.4390
Epoch 32/100
323362/323362 [==============================] - 11s 33us/step - loss: 2369.6142 - mean_absolute_error: 2369.6165
Epoch 33/100
323362/323362 [==============================] - 11s 33us/step - loss: 2371.2249 - mean_absolute_error: 2371.2253
Epoch 34/100
323362/323362 [==============================] - 11s 34us/step - loss: 2365.0106 - mean_absolute_error: 2365.0117
Epoch 35/100
323362/323362 [==============================] - 11s 33us/step - loss: 2359.2482 - mean_absolute_error: 2359.2473
Epoch 36/100
323362/323362 [==============================] - 17s 53us/step - loss: 2357.6565 - mean_absolute_error: 2357.65

323362/323362 [==============================] - 18s 54us/step - loss: 2322.0492 - mean_absolute_error: 2322.0488
Epoch 54/100
323362/323362 [==============================] - 18s 56us/step - loss: 2305.1305 - mean_absolute_error: 2305.1299
Epoch 55/100
323362/323362 [==============================] - 16s 51us/step - loss: 2310.7343 - mean_absolute_error: 2310.7341
Epoch 56/100
323362/323362 [==============================] - 19s 58us/step - loss: 2308.3980 - mean_absolute_error: 2308.3977
Epoch 57/100
323362/323362 [==============================] - 18s 54us/step - loss: 2307.5959 - mean_absolute_error: 2307.5950
Epoch 58/100
323362/323362 [==============================] - 18s 56us/step - loss: 2297.0647 - mean_absolute_error: 2297.0662
Epoch 59/100
323362/323362 [==============================] - 18s 56us/step - loss: 2301.4244 - mean_absolute_error: 2301.4233
Epoch 60/100
323362/323362 [==============================] - 18s 55us/step - loss: 2299.6601 - mean_absolute_error: 2299.65

323362/323362 [==============================] - 19s 58us/step - loss: 2257.4622 - mean_absolute_error: 2257.4622
Epoch 82/100
323362/323362 [==============================] - 19s 58us/step - loss: 2273.8373 - mean_absolute_error: 2273.8381
Epoch 83/100
323362/323362 [==============================] - 18s 55us/step - loss: 2263.8263 - mean_absolute_error: 2263.8245
Epoch 84/100
323362/323362 [==============================] - 17s 54us/step - loss: 2264.8919 - mean_absolute_error: 2264.8916
Epoch 85/100
323362/323362 [==============================] - 18s 57us/step - loss: 2258.3618 - mean_absolute_error: 2258.3611
Epoch 86/100
323362/323362 [==============================] - 18s 55us/step - loss: 2266.6218 - mean_absolute_error: 2266.6226
Epoch 87/100
323362/323362 [==============================] - 18s 56us/step - loss: 2269.1113 - mean_absolute_error: 2269.1099
Epoch 88/100
323362/323362 [==============================] - 16s 50us/step - loss: 2260.0799 - mean_absolute_error: 2260.07

323362/323362 [==============================] - 18s 57us/step - loss: 2717.8388 - mean_absolute_error: 2717.8384
Epoch 8/100
323362/323362 [==============================] - 20s 63us/step - loss: 2691.2149 - mean_absolute_error: 2691.2119
Epoch 9/100
323362/323362 [==============================] - 20s 62us/step - loss: 2651.1221 - mean_absolute_error: 2651.1218
Epoch 10/100
323362/323362 [==============================] - 18s 54us/step - loss: 2628.7251 - mean_absolute_error: 2628.7251
Epoch 11/100
323362/323362 [==============================] - 21s 65us/step - loss: 2588.5987 - mean_absolute_error: 2588.5999
Epoch 12/100
323362/323362 [==============================] - 20s 63us/step - loss: 2552.2944 - mean_absolute_error: 2552.2957
Epoch 13/100
323362/323362 [==============================] - 19s 58us/step - loss: 2530.9267 - mean_absolute_error: 2530.9263
Epoch 14/100
323362/323362 [==============================] - 20s 62us/step - loss: 2511.1541 - mean_absolute_error: 2511.1541

323362/323362 [==============================] - 18s 57us/step - loss: 2375.8592 - mean_absolute_error: 2375.8586
Epoch 36/100
323362/323362 [==============================] - 19s 59us/step - loss: 2365.5763 - mean_absolute_error: 2365.5762
Epoch 37/100
323362/323362 [==============================] - 19s 60us/step - loss: 2357.0558 - mean_absolute_error: 2357.0579
Epoch 38/100
323362/323362 [==============================] - 16s 51us/step - loss: 2352.0558 - mean_absolute_error: 2352.0562
Epoch 39/100
323362/323362 [==============================] - 19s 60us/step - loss: 2367.3619 - mean_absolute_error: 2367.3640
Epoch 40/100
323362/323362 [==============================] - 19s 59us/step - loss: 2337.2421 - mean_absolute_error: 2337.2424
Epoch 41/100
323362/323362 [==============================] - 18s 54us/step - loss: 2351.5735 - mean_absolute_error: 2351.5742
Epoch 42/100
323362/323362 [==============================] - 18s 55us/step - loss: 2350.0946 - mean_absolute_error: 2350.09

Epoch 63/100
323362/323362 [==============================] - 18s 57us/step - loss: 2289.4630 - mean_absolute_error: 2289.4646
Epoch 64/100
323362/323362 [==============================] - 18s 56us/step - loss: 2286.9976 - mean_absolute_error: 2286.9985
Epoch 65/100
323362/323362 [==============================] - 18s 56us/step - loss: 2288.8456 - mean_absolute_error: 2288.8467
Epoch 66/100
323362/323362 [==============================] - 17s 54us/step - loss: 2275.5492 - mean_absolute_error: 2275.5483
Epoch 67/100
323362/323362 [==============================] - 18s 57us/step - loss: 2283.0176 - mean_absolute_error: 2283.0186
Epoch 68/100
323362/323362 [==============================] - 21s 65us/step - loss: 2288.6811 - mean_absolute_error: 2288.6821
Epoch 69/100
323362/323362 [==============================] - 20s 62us/step - loss: 2286.0341 - mean_absolute_error: 2286.0352
Epoch 70/100
323362/323362 [==============================] - 19s 60us/step - loss: 2275.4526 - mean_absolute_e

Epoch 27/100
323362/323362 [==============================] - 18s 56us/step - loss: 2394.9761 - mean_absolute_error: 2394.9775
Epoch 28/100
323362/323362 [==============================] - 19s 57us/step - loss: 2403.8715 - mean_absolute_error: 2403.8694
Epoch 29/100
323362/323362 [==============================] - 19s 60us/step - loss: 2378.7114 - mean_absolute_error: 2378.7144
Epoch 30/100
323362/323362 [==============================] - 20s 62us/step - loss: 2378.9093 - mean_absolute_error: 2378.9097
Epoch 31/100
323362/323362 [==============================] - 18s 56us/step - loss: 2375.9592 - mean_absolute_error: 2375.9575
Epoch 32/100
323362/323362 [==============================] - 19s 60us/step - loss: 2374.9418 - mean_absolute_error: 2374.9397
Epoch 33/100
323362/323362 [==============================] - 21s 64us/step - loss: 2386.1537 - mean_absolute_error: 2386.1514
Epoch 34/100
323362/323362 [==============================] - 18s 56us/step - loss: 2378.9345 - mean_absolute_e

323362/323362 [==============================] - 23s 71us/step - loss: 2315.4291 - mean_absolute_error: 2315.4282
Epoch 55/100
323362/323362 [==============================] - 20s 62us/step - loss: 2334.0829 - mean_absolute_error: 2334.0833
Epoch 56/100
323362/323362 [==============================] - 25s 76us/step - loss: 2326.2275 - mean_absolute_error: 2326.2278
Epoch 57/100
323362/323362 [==============================] - 21s 65us/step - loss: 2305.2418 - mean_absolute_error: 2305.2415
Epoch 58/100
323362/323362 [==============================] - 22s 69us/step - loss: 2326.0578 - mean_absolute_error: 2326.0574
Epoch 59/100
323362/323362 [==============================] - 25s 76us/step - loss: 2306.2856 - mean_absolute_error: 2306.2839 6s - loss: 2325.3078 - ETA: 0s - loss: 2304.9463 - mean_absolute_error: 2304
Epoch 60/100
323362/323362 [==============================] - 21s 64us/step - loss: 2314.6965 - mean_absolute_error: 2314.6965
Epoch 61/100
323362/323362 [===================

In [18]:
media = resultados.mean()
desvio = resultados.std()

In [19]:
print(media, '\n', desvio)

-2290.0939126190797 
 70.30969702739353
